In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import requests
import os

In [3]:
file = 'tiny_shakespere.txt'
if not os.path.isfile(file):
    response = requests.get("https://raw.githubusercontent.com/jcjohnson/torch-rnn/master/data/tiny-shakespeare.txt")
    response.raise_for_status()
    with open(file, 'w') as f:
        f.write(response.text)

shakespere = open(file, 'r').read()
chars = sorted(list(set(shakespere)))
vocab_size = len(chars)

print(len(shakespere))
print(chars)
print(vocab_size)

1115394
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


In [4]:
itos = dict(zip(range(len(chars)), chars))
stoi = dict(zip(chars, range(len(chars))))
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hello World!"))
print(decode(encode("Hello World!")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 2]
Hello World!


In [5]:
data = torch.tensor(encode(shakespere), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:50])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56])


In [6]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('x:', xb.shape, xb)
print('y:', yb.shape, yb)

print('------')

for b in range (batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print("inputs", context.tolist(), 'expects', target.tolist())

x: torch.Size([4, 8]) tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
y: torch.Size([4, 8]) tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------
inputs [24] expects 43
inputs [24, 43] expects 58
inputs [24, 43, 58] expects 5
inputs [24, 43, 58, 5] expects 57
inputs [24, 43, 58, 5, 57] expects 1
inputs [24, 43, 58, 5, 57, 1] expects 46
inputs [24, 43, 58, 5, 57, 1, 46] expects 43
inputs [24, 43, 58, 5, 57, 1, 46, 43] expects 39
inputs [44] expects 53
inputs [44, 53] expects 56
inputs [44, 53, 56] expects 1
inputs [44, 53, 56, 1] expects 58
inputs [44, 53, 56, 1, 58] expects 46
inputs [44, 53, 56, 1, 58, 46] expects 39
inputs [44, 53, 56, 1, 58, 46, 39] expects 58
inputs [44, 53, 56, 1, 58, 46, 39, 58] expects 1
inputs [52] expects 58
inputs [52, 5

In [8]:
class BigramLangageModel(nn.Module):
    def __init__(self, vocab_size, n_emb) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
    
    def forward(self, idx, targets = None):
        logits = self.token_embedding_table(idx) # (B, T, C) - Batch, Time, Channels(Embedding)

        if targets is None:
            loss = None
        else:
            # cross entropy needs logits as (B*T, C) and target as (B*T)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array os indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLangageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss.item())
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
5.036386013031006

l-QYjt'CL?jLDuQcLzy'RIo;'KdhpV
vLixa,nswYZwLEPS'ptIZqOZJ$CA$zy-QTkeMk x.gQSFCLg!iW3fO!3DGXAqTsq3pdgq


In [9]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [10]:
batch_size = 32
for steps in range(1000):
    # sample batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


3.6379246711730957


In [11]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


N3vVoesMyas:Iocindad.e-NNSqYPso&bFho&$;BQ$dZTMf'mKlf;DRPm'W,esPzyXAzCA$;GunqCEy&Oy;ZxjKVhmrdhxCAbTSp-Rue.HQN
!GW&kwst-s
OMc.hpENUZEzlda$BPPtIhe!,epdheaRns:
AqW3E,DXU,NENT n b.u.xIYV&j'nnl;s,ngtoOm ixPLenrXElrPjIU-T'SmCPJ
cra3bLGhT ALM-veAYkpr ,erPVhJQNV
P?WN3b?oYxpig;ENTy3q&j.mes; iZ,w..w&yEK
Ona$Iy


In [17]:
# SelfAttention Trick
B, T, C = 4, 8, 32 # Batch, Time, Channels
x = torch.randn(B, T, C)
# print(x.shape)

# # We want x[b,t] = mean_{i<=t} x[b,i]
# xbow2 = torch.zeros((B,T,C))
# for b in range(B):
#     for t in range(T):
#         xprev = x[b, :t+1] # (t,C)
#         xbow2[b, t] = torch.mean(xprev, 0)

# # a = torch.tril(torch.ones(3, 3))
# # a = a / torch.sum(a, 1, keepdim=True)
# # b = torch.randint(0, 10,(3,2)).float()
# # c = a @ b
# # a, b, c

# let's see a single Head perform self_attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, head_size)
q = query(x) # (B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5 # (B, T, head_size) @ (B, head_size, T) = (B, T, T)

tril = torch.tril(torch.ones((T, T)))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
xbow = wei @ v
# torch.allclose(xbow, xbow2)

print(xbow.shape)

torch.Size([4, 8, 16])
